In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.utils import to_categorical
from skimage import io
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from matplotlib import ticker
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import random

#train_dataset loading
input_paths = []
y = []
base_dir = "path"
for f in sorted(os.listdir(base_dir)):
    if f == 'background':
        continue
    dir_path = os.path.join(base_dir, f)
    if os.path.isdir(dir_path):
        for i in sorted(os.listdir(dir_path)):
            input_paths.append(os.path.join(dir_path, i))
            y.append(f)
input_paths, y

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def show_values(pc, fmt="%.2f", **kw):
    pc.update_scalarmappable()
    ax = pc.axes
    #ax = pc.axes# FOR LATEST MATPLOTLIB
    #Use zip BELOW IN PYTHON 3
    for p, color, value in zip(pc.get_paths(), pc.get_facecolors(), pc.get_array()):
        x, y = p.vertices[:-2, :].mean(0)
        if np.all(color[:3] > 0.5):
            color = (0.0, 0.0, 0.0)
        else:
            color = (1.0, 1.0, 1.0)
        ax.text(x, y, fmt % value, ha="center", va="center", color=color, **kw)


def cm2inch(*tupl):
    inch = 2.54
    if type(tupl[0]) == tuple:
        return tuple(i/inch for i in tupl[0])
    else:
        return tuple(i/inch for i in tupl)


def heatmap(AUC, title, xlabel, ylabel, xticklabels, yticklabels, figure_width=40, figure_height=20, correct_orientation=False, cmap='RdBu'):
    # Plot it out
    fig, ax = plt.subplots()    
    #c = ax.pcolor(AUC, edgecolors='k', linestyle= 'dashed', linewidths=0.2, cmap='RdBu', vmin=0.0, vmax=1.0)
    c = ax.pcolor(AUC, edgecolors='k', linestyle= 'dashed', linewidths=0.2, cmap=cmap)

    # put the major ticks at the middle of each cell
    ax.set_yticks(np.arange(AUC.shape[0]) + 0.5, minor=False)
    ax.set_xticks(np.arange(AUC.shape[1]) + 0.5, minor=False)

    # set tick labels
    #ax.set_xticklabels(np.arange(1,AUC.shape[1]+1), minor=False)
    ax.set_xticklabels(xticklabels, minor=False)
    ax.set_yticklabels(yticklabels, minor=False)

    # set title and x/y labels
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)      

    # Remove last blank column
    plt.xlim( (0, AUC.shape[1]) )

    # Turn off all the ticks
    ax = plt.gca()    
    for t in ax.xaxis.get_major_ticks():
        t.tick1On = False
        t.tick2On = False
    for t in ax.yaxis.get_major_ticks():
        t.tick1On = False
        t.tick2On = False

    # Add color bar
    plt.colorbar(c)

    # Add text in each cell 
    show_values(c)

    # Proper orientation (origin at the top left instead of bottom left)
    if correct_orientation:
        ax.invert_yaxis()
        ax.xaxis.tick_top()       

    # resize 
    fig = plt.gcf()
    #fig.set_size_inches(cm2inch(40, 20))
    #fig.set_size_inches(cm2inch(40*4, 20*4))
    fig.set_size_inches(cm2inch(figure_width, figure_height))



def plot_classification_report(classification_report, title='Classification report ', cmap='RdBu'):
    lines = classification_report.split('\n')

    classes = []
    plotMat = []
    support = []
    class_names = []

    for line in lines[2 : (len(lines) - 4)]:
        t = line.strip().split()
        if len(t) < 2: continue
        classes.append(t[0])
        v = [float(x) for x in t[1: len(t) - 1]]
        support.append(int(t[-1]))
        class_names.append(t[0])
        print(v)
        plotMat.append(v)

    print('plotMat: {0}'.format(plotMat))
    print('support: {0}'.format(support))

    xlabel = 'Metrics'
    ylabel = 'Classes'
    xticklabels = ['Precision', 'Recall', 'F1-score']
    yticklabels = ['{0} ({1})'.format(class_names[idx], sup) for idx, sup  in enumerate(support)]
    figure_width = 25
    figure_height = len(class_names) + 7
    correct_orientation = False
    heatmap(np.array(plotMat), title, xlabel, ylabel, xticklabels, yticklabels, figure_width, figure_height, correct_orientation, cmap=cmap)

In [ ]:
X = []
for img_path in input_paths:
    img = io.imread(img_path, as_gray=True)
    img = resize(img, (64, 64),
                       anti_aliasing=True)
    X.append(img)


In [ ]:
X_test =  X
X_test = np.expand_dims(X_test, 3)
y_test = y#true values

temp = []

for data in y_test:
    if data == 'five':
        temp.append(5)
    if data == 'four':
        temp.append(4)
    if data == 'three':
        temp.append(3)
    if data == 'two':
        temp.append(2)
    if data == 'one':
        temp.append(1)
    if data == 'zero':
        temp.append(0)
    if data == 'eleven':
        temp.append(11)
    if data == 'ten':
        temp.append(10)
    if data == 'nine':
        temp.append(9)
    if data == 'eight':
        temp.append(8)
    if data == 'seven':
        temp.append(7)
    if data == 'six':
        temp.append(6)

y_test = temp

y_test = to_categorical(y_test, num_classes=12)

In [ ]:
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report

model = load_model('freshDatasetModel.h5')

class_labels = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven']
class_labels = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven']
predictions = model.predict(X_test)
y_pred = predictions.argmax(1)

y_true = y_test.argmax(1)

report = classification_report(y_true, y_pred, target_names=class_labels)
print(report)

plt.show()

In [ ]:
plot_classification_report(report)
plt.savefig('classification_report_different_hands.png', dpi=100, format='png', bbox_inches='tight')